In [ ]:
import json
import boto3

import sagemaker
from sagemaker import get_execution_role
from sagemaker.s3 import S3Uploader, S3Downloader

In [ ]:
region = boto3.Session().region_name
role = get_execution_role()
bucket = sagemaker.Session().default_bucket()
prefix = "phi-masking"
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [ ]:
documentName = "health_notes.png"

In [ ]:
client = boto3.client(service_name='textract', region_name='eu-west-2',
                     endpoint_url='https://textract.eu-west-2.amazonaws.com')

In [ ]:
with open(documentName, 'rb') as file:
    img_file = file.read()
    bytes_arr = bytearray(img_file)
    print('Image file is loaded', documentName)

In [ ]:
response = client.detect_document_text(Document={'Bytes': bytes_arr})

In [ ]:
!pip install amazon-textract-response-parser

In [ ]:
from trp import Document
doc = Document(response)
page_string = ''
for page in doc.pages:
    for line in page.lines:
        page_string += " "
        page_string += str(line.text)
print(page_string)

In [ ]:
text_data = 'health_notes.txt'
doc = Document(response)
with open(text_data, 'w', encoding='utf-8') as f:
    for page in doc.pages:
        page_string = ''
        for line in page.lines:
            page_string += " "
            page_string += str(line.text)
        f.writelines(page_string + "\n")

In [ ]:
with open(text_data, "r") as fi:
    raw_texts = [line.strip() for line in fi.readlines()]

In [ ]:
s3 = boto3.resource('s3')
s3.Bucket(bucket).upload_file("health_notes.txt", "phi-masking/health_notes.txt")

In [ ]:
comprehendMedical = boto3.client(service_name='comprehendmedical')
entities = comprehendMedical.detect_entities_v2(Text=page_string)

In [ ]:
print(entities)

In [ ]:
page_string

In [ ]:
substrings = []
start = 0
for entity in piilist["Entities"]:
    substrings.append(page_string[start:entity["BeginOffset"]])
    substrings.append(entity["Type"])
    start = entity["EndOffset"]

masked_text = " ".join(substrings)

In [ ]:
print(masked_text)